In [1]:
from transformers import AutoConfig
import numpy as np
import torch
from torch.utils.data import DataLoader, Subset
from vicuna_llava import vicuna_llava, dataset_llava
from accelerate import Accelerator
accelerator = Accelerator()

Log in to huggingface (using llama requires you to request access on huggingface)

In [2]:

from huggingface_hub import login
login()

### Instantiate vicunallava

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# grab vicuna and its tokenizer
model_name = "meta-llama/Llama-3.2-1B"#"lmsys/vicuna-7b-v1.5"
config = AutoConfig.from_pretrained(model_name)

vicunallava = accelerator.prepare(vicuna_llava(config, llmURL=model_name, accelerator=accelerator))


Grab pretrained projector

In [4]:
linear_llava_proj = torch.load('Pretrained/vicunallava_im_embedding_stage1.pt')

with torch.no_grad():
    vicunallava.im_embedding.weight.copy_(linear_llava_proj.weight)
    vicunallava.im_embedding.bias.copy_(linear_llava_proj.bias)

C:\Users\colli\AppData\Local\Temp\ipykernel_23384\181095193.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  linear_llava_proj = torch.load('Pretrained/vicunallava_im_emb

### Instantiate Dataset

In [5]:
# directories pointing to images directory and chat.json, I've downloaded the dataset locally to deal with some of the missing images
chat = 'CC3M/chat.json'
im_dir = "CC3M/images/"

num_samples=50
subset_idcs = list(range(num_samples))

cc3m_dataset = accelerator.prepare(Subset(dataset_llava(chat, im_dir), subset_idcs))
batch_size=10
cc3m_dataloader =  accelerator.prepare(DataLoader(cc3m_dataset, batch_size=batch_size))

Test Generating on Some Sample from Dataset

In [6]:
from PIL import Image
from torchvision.transforms import ToPILImage, ToTensor
ToPIL = ToPILImage()


# grab image from dataset
#testprompt, testimage, testresponse = cc3m_dataset[0]
#testimage = testimage.to(device)


# grab a loaded image
#testprompt = "Describe the image"
#image = Image.open("Forza Horizon 5 Monaro Profile Clean.jpg")
#testimage = ToTensor()(image)
#testresponse = "A car"

#output = vicunallava.generate(testimage, testprompt, max_new_tokens=15)
#print(f'input prompt: {testprompt}')

#ToPIL(testimage).show()

#print(f'response: {output}')
#print(f'ground truth: {testresponse}')

### Training (Stage 1)

Set which parameters to be optimized

In [ ]:
from torch.optim import Adam
from transformers import get_scheduler
from tqdm.auto import tqdm



optimizer =  accelerator.prepare(Adam([{"params":vicunallava.parameters(),
                                        "params":vicunallava.im_embedding.parameters()}], lr=2e-3))


num_epochs = 1
gradient_accumulation_steps = 8
num_dataset_samples = len(cc3m_dataset)
num_batch_steps = num_epochs*num_dataset_samples/batch_size
num_optim_steps = num_batch_steps/gradient_accumulation_steps
num_warmup_steps = np.ceil(0.03*num_optim_steps)
lr_scheduler = accelerator.prepare(get_scheduler("cosine",
                             optimizer=optimizer,
                             num_warmup_steps=num_warmup_steps,
                             num_training_steps=num_batch_steps))


vicunallava.staged_training(cc3m_dataloader,1,num_epochs,batch_size,8,optimizer,lr_scheduler,save_results=False,push_to_hub=False,hf_user="cjschul")



  0%|          | 0/5 [00:00<?, ?it/s]

c:\Users\colli\OneDrive\Documents\Classes\UofM\ECE 598 (LLMs)\Final Project\llava_finalproject\llava_finalproject\vicuna_llava.py:105: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


batchiter:1, loss:1.4420560598373413
batchiter:2, loss:1.4395617246627808
batchiter:3, loss:1.434531569480896
batchiter:4, loss:1.4348585605621338
batchiter:5, loss:1.4387779235839844


model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.22G [00:00<?, ?B/s]